In [2]:
%%time
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yaml
from datetime import datetime
import time

Wall time: 10.6 s


In [3]:
%%time
alltour = 'http://liquipedia.net/dota2/Premier_Tournaments'
htmlall = requests.get(alltour)
htmlall.encoding = 'utf-8'
soup = BeautifulSoup(htmlall.text, "lxml")
divs = soup.find_all('b')
urls = []
for div in divs:
    urls.append(re.search(r'href="(.+?)"', str(div))[1])
df = pd.DataFrame(columns = ['team',
                             'player',
                             'position',
                             'tour',
                             'place',
                             'prize',
                             'start', 
                             'end', 
                             'startelo32',
                             'startelo64',
                             'startglickorating',
                             'startglickomu',
                             'startglickophi',
                             'startglickosigma',
                             'startglicko2rating',
                             'startglicko2mu',
                             'startglicko2phi',
                             'startglicko2sigma',
                             'endelo32',
                             'endelo64',
                             'endglickorating',
                             'endglickomu',
                             'endglickophi',
                             'endglickosigma',
                             'endglicko2rating',
                             'endglicko2mu',
                             'endglicko2phi',
                             'endglicko2sigma'])
df2 = pd.DataFrame(columns = ['tour',
                              'place',
                              'prize',
                              'team'])

Wall time: 1.39 s


In [25]:
%%time
for url in urls[114:]:
    try:
        url = 'http://wiki.teamliquid.net' + url
        html = requests.get(url)
        html.encoding = 'utf-8'
        soup = BeautifulSoup(html.text, "lxml")
        #find tournament name
        tournamentname = soup.find_all('span', dir="auto")
        tournamentname = tournamentname[0].contents[0]
        #find tournament prizepool
        prizedict = {}
        prizess = soup.find_all('table', class_="table table-bordered prizepooltable collapsed")
        prizes = prizess[0].find_all('tr')
        place = 0
        sameplace = 0
        for p in prizes[1:]:
            if re.search(r'[$]\w+,\w+,\w+|[$]\w+,\w+|[$]\w+', str(p)) is not None:
                prize = re.search(r'[$]\w+,\w+,\w+|[$]\w+,\w+|[$]\w+', str(p))                
                place += 1 + sameplace
                sameplace = 0
                prize = prize[0]
                team = p.find_all('a')[-1].contents[0]
                df2 = df2.append({'tour': tournamentname,
                           'place': place,
                           'prize': prize,
                           'team': team}, ignore_index=True)
                teamlower = re.sub(r'\W+', '', team).lower()
                prizedict[teamlower] = [place, prize]
            else:
                sameplace += 1
                team = p.find_all('a')[-1].contents[0]
                teamlower = re.sub(r'\W+', '', team).lower()
                prizedict[teamlower] = [place, prize]                
                df2 = df2.append({'tour': tournamentname,
                           'place': place,
                           'prize': prize,
                           'team': team}, ignore_index=True).fillna(method='ffill') 
        #find tournament info
        info = soup.find_all('div', text = re.compile(r'\d*-\d*-\d*'), class_="infobox-cell-2")
        startdate = info[0].contents[0]
        enddate = info[1].contents[0]
        #check if tournament is already played
        if datetime.strptime(enddate, '%Y-%m-%d') > datetime.now():
            continue
        startdate2 = startdate[-2:] + startdate[4:8] + startdate[:4]
        enddate2 = enddate[-2:] + enddate[4:8] + enddate[:4]
        #rating of startdate from datdota and turn into list of dict
        urldat = 'http://www.datdota.com/api/ratings?date=' + startdate2
        htmldat = requests.get(urldat)
        htmldat.encoding = 'utf-8'
        elo = BeautifulSoup(htmldat.text, "lxml")
        elodict = yaml.load(elo.text)
        startelolist = elodict['data']   
        #rating of enddate from datdota and turn into list of dict
        urldat = 'http://www.datdota.com/api/ratings?date=' + enddate2
        htmldat = requests.get(urldat)
        htmldat.encoding = 'utf-8'
        elo = BeautifulSoup(htmldat.text, "lxml")
        elodict = yaml.load(elo.text)
        endelolist = elodict['data']
        
        #divs contain all of team info in tournament
        divs = soup.find_all('div', class_="teamcard") 
        
        #for each info of team
        for div in divs:
            #find teamname
            center = div.find('center')
            teamname = re.search(r'title="(.+?)"', str(center))[1]
            teamnamelower = re.sub(r'\W+', '', teamname).lower()
            #init elo32,64
            startelo32 = 'N/A'
            startelo64 = 'N/A'
            startglickorating = 'N/A'
            startglickomu = 'N/A'
            startglickophi = 'N/A'
            startglickosigma = 'N/A'
            startglicko2rating = 'N/A'
            startglicko2mu = 'N/A'
            startglicko2sigma = 'N/A'
            endelo32 = 'N/A'
            endelo64 = 'N/A'
            endglickorating = 'N/A'
            endglickomu = 'N/A'
            endglickophi = 'N/A'
            endglickosigma = 'N/A'
            endglicko2rating = 'N/A'
            endglicko2mu = 'N/A'
            endglicko2phi = 'N/A'
            endglicko2sigma = 'N/A'
            
            #for each dict in list of dict of startelo compare teamname to teamname from liquipedia
            for startelo in startelolist:
                startelolower = re.sub(r'\W+', '', startelo['teamName'].lower())
                if (((startelolower in teamnamelower)
                or (teamnamelower in startelolower))
                and (abs(len(teamnamelower) - len(startelolower)) < 5)):
                    startelo32 = startelo['elo32']['current']
                    startelo64 = startelo['elo64']['current']
                    startglickorating = startelo['glicko']['rating']
                    startglickomu = startelo['glicko']['mu']
                    startglickophi = startelo['glicko']['phi']
                    startglickosigma = startelo['glicko']['sigma']
                    startglicko2rating = startelo['glicko2']['rating']
                    startglicko2mu = startelo['glicko2']['mu']
                    startglicko2phi = startelo['glicko2']['phi']                    
                    startglicko2sigma = startelo['glicko2']['sigma']
            for endelo in endelolist:
                endelolower = re.sub(r'\W+', '', endelo['teamName'].lower())
                if (((endelolower in teamnamelower)
                or (teamnamelower in endelolower))
                and (abs(len(teamnamelower) - len(endelolower)) < 5)):
                    endelo32 = endelo['elo32']['current']
                    endelo64 = endelo['elo64']['current']
                    endglickorating = endelo['glicko']['rating']
                    endglickomu = endelo['glicko']['mu']
                    endglickophi = endelo['glicko']['phi']
                    endglickosigma = endelo['glicko']['sigma']
                    endglicko2rating = endelo['glicko2']['rating']
                    endglicko2mu = endelo['glicko2']['mu']
                    endglicko2phi = endelo['glicko2']['phi']
                    endglicko2sigma = endelo['glicko2']['sigma']

            for table in div.find_all('table', class_="table table-bordered list"):
                for tr in table.find_all('tr'):
                    string = re.search(r'<th>([12345])</th>.+title="(\w+)"', str(tr))
                    if string is None:
                        continue
                    df = df.append({'team': teamname,
                                    'position': string[1],
                                    'player': string[2],  
                                    'tour': tournamentname, 
                                    'place': prizedict[teamnamelower][0],
                                    'prize': prizedict[teamnamelower][1],
                                    'start': startdate, 
                                    'end': enddate, 
                                    'startelo32': startelo32,
                                    'startelo64': startelo64,
                                    'startglickorating': startglickorating,
                                    'startglickomu': startglickomu,
                                    'startglickophi': startglickophi,
                                    'startglickosigma': startglickosigma,
                                    'startglicko2rating': startglicko2rating,
                                    'startglicko2mu': startglicko2mu,
                                    'startglicko2phi': startglicko2phi,
                                    'startglicko2sigma': startglicko2sigma,
                                    'endelo32': endelo32,
                                    'endelo64': endelo64,
                                    'endglickorating': endglickorating,
                                    'endglickomu': endglickomu,
                                    'endglickophi': endglickophi,
                                    'endglickosigma': endglickosigma,
                                    'endglicko2rating': endglicko2rating,
                                    'endglicko2mu': endglicko2mu,
                                    'endglicko2phi': endglicko2phi,
                                    'endglicko2sigma': endglicko2sigma}, ignore_index=True)  
        time.sleep(3)
    except IndexError:
        continue
        
    except NameError:
        continue
        
    except TypeError:
        continue
        
    except KeyError:
        continue

Wall time: 2min


In [27]:
df2

,tour,place,prize,team
0,ESL One Genting 2018,1,"$160,000","<img alt="""" height=""24"" src=""/commons/images/1..."
1,DreamLeague Season 8,1,"$500,000",Team Secret
2,DreamLeague Season 8,2,"$200,000",Team Liquid
3,DreamLeague Season 8,3,"$100,000",Evil Geniuses
4,DreamLeague Season 8,4,"$70,000",Natus Vincere
5,DreamLeague Season 8,5,"$45,000",Newbee
6,DreamLeague Season 8,5,"$45,000",Virtus.pro
7,DreamLeague Season 8,7,"$20,000",Fnatic
8,DreamLeague Season 8,7,"$20,000",Infamous
9,ESL One Hamburg 2017,1,"$500,000",Virtus.pro


In [22]:
df2.groupby('place').size()

place
1     109
2     108
3     172
4     204
5     237
6       6
7     130
8       2
9      56
11      2
13     32
17      6
dtype: int64

In [24]:
urls[113:]

['/dota2/DreamHack/2013/Summer',
 '/dota2/G-1_Champions_League_Season_5',
 '/dota2/The_Premier_League/Season_4',
 '/dota2/ESL_Major_Series_One/Spring/Main_Event',
 '/dota2/RaidCall_Dota_2_League_Season_2',
 '/dota2/StarLadder/StarSeries/Season_5',
 '/dota2/The_Defense/Season_3',
 '/dota2/G-League_2012_Season_2',
 '/dota2/The_Premier_League/Season_3',
 '/dota2/StarLadder/StarSeries/Season_4',
 '/dota2/The_Asia_2012',
 '/dota2/ASUS_Open_2012',
 '/dota2/Thor_Open_2012',
 '/dota2/World_Cyber_Games_2012',
 '/dota2/DreamHack/2012/Winter',
 '/dota2/G-1_Champions_League_Season_4',
 '/dota2/RaidCall_Dota_2_League_Season_1',
 '/dota2/Electronic_Sports_World_Cup_2012',
 '/dota2/StarLadder/StarSeries/Season_3',
 '/dota2/The_International/2012',
 '/dota2/The_Defense/Season_2',
 '/dota2/ProDOTA2_League/Season_1/Pro',
 '/dota2/StarLadder/StarSeries/Season_2',
 '/dota2/The_Premier_League/Season_2',
 '/dota2/DreamHack/2012/Summer',
 '/dota2/StarLadder/StarSeries/Season_1',
 '/dota2/The_Premier_League/S

In [29]:
df.to_csv('premierwithprize.csv')

In [21]:
urls[97:]

['/dota2/V_Games_Tournament',
 '/dota2/WVW_National_Electronic_Sports_Classic/June',
 '/dota2/GEST_The_Challenge_June',
 '/dota2/CDEC_New_Star_Challenge',
 '/dota2/Korea_Dota_2_League/Season_2/Tier_1',
 '/dota2/ECL_2014/Season_1',
 '/dota2/American_Dota_League/Season_2',
 '/dota2/Techlabs_Cup/2014/Season_2',
 '/dota2/WVW_National_Electronic_Sports_Classic/April',
 '/dota2/DreamHack/2014/Bucharest',
 '/dota2/Fragbite_Masters_2014',
 '/dota2/MLG_T.K.O._Tournament/Europe',
 '/dota2/Korea_Dota_2_League/Season_1',
 '/dota2/MLG_T.K.O._Tournament/America',
 '/dota2/Techlabs_Cup/2014/Season_1',
 '/dota2/Captains_Draft/1',
 '/dota2/Netolic_Pro_League_5/East',
 '/dota2/Korea_Dota_2_League/Season_1/Tier_1',
 '/dota2/Monster_Energy_Invitational',
 '/dota2/Sina_Cup_Supernova_Dota_2_Open/Season_3',
 '/dota2/Electronic_Sports_Prime/Shock_Therapy_Cup',
 '/dota2/Pinnacle_Pro-Am_Cup',
 '/dota2/Asian_Cyber_Games_2013',
 '/dota2/Fengyun_Dota_2_Tournament',
 '/dota2/Fragbite_Masters_2013',
 '/dota2/Netolic